# Features

1. Vector storage
2. Retriver
3. Chat engine
4. Memory system
5. Conversational query 
6. Conversational query with memory

In [3]:
import pandas as pd
from datetime import datetime

from IPython.display import display
from IPython.display import Markdown

import google.generativeai as genai

from utils import read_config, OracleAgent

from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.gemini import Gemini

import chromadb

c:\Users\sean.chang\AppData\Local\anaconda3\envs\bibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
info_path = '.env/info.json'
info = read_config(info_path)

oracle_agent = OracleAgent(info['DW_conn_info'])

API_KEY= info['gkey']
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')


In [6]:
query = """
select rowid, dashboard_name, report_name, page_name, pic_name, llm, llm_question 
from bi_dashboard
"""

df_dashboard = oracle_agent.read_table(query=query)
# df_dashboard.to_csv('opt/dashboard_data.csv', index=False, encoding='utf-16')

In [4]:
# df_dashboard = pd.read_csv('files/dashboard_data.csv', encoding='utf-16')

In [7]:
df_dashboard = df_dashboard.rename(columns={"llm": "description"})

In [8]:
df_dashboard.head()

,rowid,dashboard_name,report_name,page_name,pic_name,description,llm_question
0,AAi0soABZAAAACLAAC,投控經營分析,華紙-營運績效分析月報,紙機生產狀況,紙機生產狀況-得率趨勢圖.jpg,這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率...,這張圖片顯示了華紙在2024年1月到5月的紙機生產狀況，以下列出10個關於這張圖片的問題：\...
1,AAi0soABZAAAACLAAF,投控經營分析,華紙-營運績效分析月報,號機直接法損益,號機直接法損益-完整畫面.jpg,這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本...,這張圖片顯示了華紙的營運績效分析月報，可以回答以下問題：\n\n1. **華紙的總成本是多少...
2,AAi0soABZAAAACOAAB,投控經營分析,華紙-營運績效分析日報,工廠面,工廠面3.JPG,這張圖片包含報表，包含投控經營分析華紙-營運績效分析日報工廠面的一些相關分析指標。\n\n*...,這張圖片顯示的是工廠生產相關的數據，可以提出以下問題：\n\n1. 今日生產的紙張總量是多...
3,AAi0soABZAAAACMAAE,投控經營分析,華紙-營運績效分析月報,銷售面,銷售面-事故金額狀況.jpg,這張圖片顯示了**華紙-營運績效分析月報銷售面**的數據，以柱狀圖的形式呈現。 \n\n**...,這張圖片顯示的是華紙-營運績效分析月報銷售面的數據，可以回答以下問題：\n\n1. 2024...
4,AAi0soABZAAAACUAAA,投控經營分析,華紙-營運績效分析月報,漿線生產狀況,漿線生產狀況-停機時間.jpg,"這張圖片只顯示了一項指標：**停機時間**，單位為分鐘，數值为50,750。\n\n如果需要...",這張圖片只顯示一個標題「停機時間（Min）」和一個單純的藍色圓形，以及下方顯示「非計劃停機 ...


In [17]:
df_dashboard = df_dashboard.drop(['llm_question'], axis=1)

In [18]:
Markdown(df_dashboard[df_dashboard['rowid'] == 'AAi0soABZAAAACUAAH'].description.values[0])

這張圖表顯示了生產製造成本的組成，可以帮助你了解不同項目在生產成本中所占的比例。

**主要分析指標：**

* **生產製造成本的總金額：** 顯示了總的生產製造成本。
* **各項成本的金額：** 顯示了不同項目在生產成本中的具體金額。
* **各項成本的百分比：** 顯示了不同項目在生產成本中所占的比例。 
* **累積百分比：** 顯示了累積的成本比例，可以帮助你快速了解不同項目的成本佔比。

**可以查詢的指標：**

* **各項成本的金額和百分比：** 您可以查看不同項目的成本金額和百分比，例如：
    * **原料：** 顯示了原材料的成本。
    * **電力：** 顯示了電力的成本。
    * **製造成本：** 顯示了生產過程中的成本。
    * **人工：** 顯示了人工成本。
    * **其他：** 顯示了其他成本。
* **累積百分比：** 您可以查看累積百分比，例如：
    * **原料的累積百分比：** 顯示了原材料成本佔總成本的比例。
    * **原料和電力的累積百分比：** 顯示了原材料和電力成本佔總成本的比例。
    * **所有成本的累積百分比：** 顯示了所有成本佔總成本的比例，應該為 100%。

**用途：**

這張圖表可以帮助您：

* 了解生產製造成本的組成。
* 分析不同項目的成本佔比。
* 找出成本的主要來源。
* 評估成本控制的成效。

**注意事項：**

* 圖表中顯示的數據是特定時間點的數據，可能不代表長期趨勢。
* 圖表中沒有提供成本變化的趨勢信息。
* 圖表中沒有提供成本控制措施的信息。


In [10]:
Markdown(df_dashboard[df_dashboard['rowid'] == 'AAi0soABZAAAACLAAF'].description.values[0])

這張圖片包含了一份營運績效分析月報，其中包含以下指標：

**成本**:
* 銷貨成本
* 原物料成本
* 燃料
* 其他成本

**銷售**:
* 銷貨收入
* 營業成本
* 營業毛利
* 營業費用

**費用**:
* 製造費用
* 銷售費用
* 管理費用

**淨利**:
* 稅前淨利
* 稅後淨利

**其他**:
* 損益表
* 損益
* 銷售額
* 銷貨量
* 存貨
* 應收帳款
* 應付帳款
* 固定資產
* 流動資產
* 負債
* 股東權益

這些指標可以幫助使用者了解公司在特定時期的財務狀況和營運績效。


In [5]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbedding(model_name=model_name)

In [5]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("DashboardQA")

In [13]:
def df_to_documents(df):
    return [Document(text=row['description'], metadata={col: row[col] for col in df.columns if col != 'descriptoin'}) for _, row in df.iterrows()]

In [19]:
documents = df_to_documents(df_dashboard)

In [20]:
documents

[Document(id_='b5244c3c-4996-447f-a028-f9bd8f3b1f20', embedding=None, metadata={'rowid': 'AAi0soABZAAAACLAAC', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '紙機生產狀況', 'pic_name': '紙機生產狀況-得率趨勢圖.jpg', 'description': '這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率百分比。 \n\n您可以查詢：\n\n- **效率**（以百分比顯示）： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **月份**： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **2024 年**： 圖表顯示了 2024 年的數據。\n\n圖表中缺少 6 月份的數據。'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='這張圖表顯示的是 2024 年 1 月到 5 月的月度效率數據。圖表上的數字代表各個月的效率百分比。 \n\n您可以查詢：\n\n- **效率**（以百分比顯示）： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **月份**： 圖表顯示了每個月的效率，從 1 月到 5 月。\n- **2024 年**： 圖表顯示了 2024 年的數據。\n\n圖表中缺少 6 月份的數據。', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='b65208c2-dcc2-48b1-be0a-5ed3146de155', embedding=None, metadata={'rowid': 'AAi0soABZAAAACLAAF', 'd

In [21]:
# store in vectordb
Settings.chunk_size = 1024
Settings.chunk_overlap = 50
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    embed_model=embeddings
)

In [12]:
# read from vectodb

# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("DashboardQA")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embeddings,
)


# Query Data from the persisted index
query_engine = index.as_retriever(similarity_top_k=3)
response = query_engine.retrieve("""               
包含
淨利:
稅前淨利
稅後淨利
 """)
display(Markdown(f"{response[0]}"))

Node ID: 0d2a5a4a-268f-4b85-b1c3-52da1e659b01
Text: 這張圖片包含了一份營運績效分析月報，其中包含以下指標：  **成本**: * 銷貨成本 * 原物料成本 * 燃料 * 其他成本
**銷售**: * 銷貨收入 * 營業成本 * 營業毛利 * 營業費用  **費用**: * 製造費用 * 銷售費用 * 管理費用
**淨利**: * 稅前淨利 * 稅後淨利  **其他**: * 損益表 * 損益 * 銷售額 * 銷貨量 * 存貨 * 應收帳款 *
應付帳款 * 固定資產 * 流動資產 * 負債 * 股東權益  這些指標可以幫助使用者了解公司在特定時期的財務狀況和營運績效。
Score:  0.198


In [7]:
response[0].node.metadata['page_name']

'號機直接法損益'

In [8]:
response

[NodeWithScore(node=TextNode(id_='0d2a5a4a-268f-4b85-b1c3-52da1e659b01', embedding=None, metadata={'rowid': 'AAi0soABZAAAACLAAF', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '號機直接法損益', 'pic_name': '號機直接法損益-完整畫面.jpg', 'description': '這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本\n* 原物料成本\n* 燃料\n* 其他成本\n\n**銷售**:\n* 銷貨收入\n* 營業成本\n* 營業毛利\n* 營業費用\n\n**費用**:\n* 製造費用\n* 銷售費用\n* 管理費用\n\n**淨利**:\n* 稅前淨利\n* 稅後淨利\n\n**其他**:\n* 損益表\n* 損益\n* 銷售額\n* 銷貨量\n* 存貨\n* 應收帳款\n* 應付帳款\n* 固定資產\n* 流動資產\n* 負債\n* 股東權益\n\n這些指標可以幫助使用者了解公司在特定時期的財務狀況和營運績效。\n'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b65208c2-dcc2-48b1-be0a-5ed3146de155', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'rowid': 'AAi0soABZAAAACLAAF', 'dashboard_name': '投控經營分析', 'report_name': '華紙-營運績效分析月報', 'page_name': '號機直接法損益', 'pic_name': '號機直接法損益-完整畫面.jpg', 'description': '這張圖片包含了一份營運績效分析月報，其中包含以下指標：\n\n**成本**:\n* 銷貨成本\n* 原